In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import Latex
from sympy import * 
from iminuit import Minuit
import seaborn as sns
import pandas as pd
import scipy as sp
import sys                                             # Modules to see files and folders in directories
sys.path.append('External_Functions')
from ExternalFunctions import Chi2Regression
from ExternalFunctions import nice_string_output, add_text_to_ax # useful functions to print fit results on figure
from scipy.stats import binom, poisson, norm
from simple_pendulum_functions import Residual_time
from Universal_functions import weighted_mean,uncertainty_weighted

In [2]:

def lprint(*args,**kwargs):
    """Pretty print arguments as LaTeX using IPython display system 
    
    Parameters
    ----------
    args : tuple 
        What to print (in LaTeX math mode)
    kwargs : dict 
        optional keywords to pass to `display` 
    """
    display(Latex('$$'+' '.join(args)+'$$'),**kwargs)

In [3]:
Tan=np.loadtxt("../data/pendulum/timer_output1_tan.dat")
Michael1=np.loadtxt("../data/pendulum/trash_data_Michael.dat")
Michael2=np.loadtxt("../data/pendulum/better_data_Michael.dat")
Chris=np.loadtxt("../data/pendulum/chris_data_0.dat")
Luca=np.loadtxt("../data/pendulum/timer_output_luca.dat")

In [4]:
def fit_function_Lin(x, slope, intercept):
    return intercept + slope*x

In [5]:
Time_lib={"Tan":Tan[1:,1],"Michael1":Michael1[:,1]-min(Michael1[:,1]),"Michael2":Michael2[1:51,1],"Chris":Chris[:,1],"Luca":Luca[:,1]}
Number_lib={"Tan":Tan[1:,0],"Michael1":Michael1[:,0],"Michael2":Michael2[1:51,0],"Chris":Chris[:,0],"Luca":Luca[:,0]}
Time=pd.DataFrame(Time_lib)
Number=pd.DataFrame(Number_lib)

In [6]:
Residual_time1={"Tan":Residual_time(Time.Tan,Number.Tan),"Michael1":Residual_time(Time.Michael1,Number.Michael1),"Michael2":Residual_time(Time.Michael2,Number.Michael2),"Chris":Residual_time(Time.Chris,Number.Chris),"Luca":Residual_time(Time.Luca,Number.Luca)}
Residual_Time_data=pd.DataFrame(Residual_time1)

In [7]:
chi2_object_Tan = Chi2Regression(fit_function_Lin, Number.Tan,Time.Tan, Time.Tan.std()) 
chi2_object_Michael1  = Chi2Regression(fit_function_Lin, Number.Michael1,Time.Michael1, Time.Michael1.std()) 
chi2_object_Michael2  = Chi2Regression(fit_function_Lin, Number.Michael2,Time.Michael2, Time.Michael2.std()) 
chi2_object_Chris  = Chi2Regression(fit_function_Lin, Number.Chris,Time.Chris, Time.Chris.std()) 
minuitLin_Tan = Minuit(chi2_object_Tan, pedantic=False, intercept=1, slope=1, print_level=0) 
minuitLin_Michael1 = Minuit(chi2_object_Michael1, pedantic=False, intercept=1, slope=1, print_level=0) 
minuitLin_Michael2 = Minuit(chi2_object_Michael2, pedantic=False, intercept=1, slope=1, print_level=0) 
minuitLin_Chris = Minuit(chi2_object_Chris, pedantic=False, intercept=1, slope=1, print_level=0) 
minuitLin_Tan.migrad();
minuitLin_Michael1.migrad(); 
minuitLin_Michael2.migrad(); 
minuitLin_Chris.migrad(); 

In [8]:

Periode=np.array([minuitLin_Tan.args[0],minuitLin_Michael1.args[0],minuitLin_Michael2.args[0],minuitLin_Chris.args[0]])
Periode_std=np.array([minuitLin_Tan.errors['slope'],minuitLin_Michael1.errors['slope'],minuitLin_Michael2.errors['slope'],minuitLin_Chris.errors['slope']])
intercept_array=np.array([minuitLin_Tan.args[1],minuitLin_Michael1.args[1],minuitLin_Michael2.args[1],minuitLin_Chris.args[1]])
intercept_std=np.array([minuitLin_Tan.errors['intercept'],minuitLin_Michael1.errors['intercept'],minuitLin_Michael2.errors['intercept'],minuitLin_Chris.errors['intercept']])
slope_weigted=weighted_mean(Periode,Periode_std)
slope_std_weigted=uncertainty_weighted(Periode_std)
intersection_weigted=weighted_mean(intercept_array,intercept_std)
intersection_std_weigted=uncertainty_weighted(intercept_std)

Rsidual_time_combined=np.block([Residual_Time_data.Tan,Residual_Time_data.Luca,Residual_Time_data.Michael1,Residual_Time_data.Michael2,Residual_Time_data.Chris])

In [ ]:

fig=plt.figure(figsize=(10,10))
ax=fig.add_axes([0,0,1,1])
ax1=fig.add_axes([0.65,0.2,0.3,0.3])
ax2 = ax.twinx()
#sns.scatterplot(x=Time.Tan,y=Residual_Time.Tan,ax=ax)
sns.scatterplot(x=Number.Tan,y=Time.Tan,ax=ax)
sns.scatterplot(x=Number.Michael1,y=Time.Michael1,ax=ax)
sns.scatterplot(x=Number.Michael2,y=Time.Michael2,ax=ax)
sns.scatterplot(x=Number.Chris,y=Time.Chris,ax=ax)
sns.scatterplot(x=Number.Luca,y=Time.Luca,ax=ax)
ax.plot(x_pend,fit_function_Lin(x_pend,slope_weigted,intersection_weigted))

sns.distplot(Rsidual_time_combined, ax=ax1)
d = {'Intercept':[intersection_weigted, intersection_std_weigted],
     'Slope':    [slope_weigted, slope_std_weigted]}
text = nice_string_output(d, extra_spacing=2, decimals=3)
add_text_to_ax(0.04, 0.95, text, ax, fontsize=20)


ax2.set_yscale('log')
fig.tight_layout()

## weighted Length of the line Length + pendilum length

In [44]:
Data=pd.read_csv("../data/data_setup.csv")

In [20]:
length_pend=weighted_mean(np.array(Data.length_pend.dropna()),np.array(Data.lengh_pend_std.dropna()))/2*10**(-2)
line_length=weighted_mean(np.array(Data.line_length.dropna()),np.array(Data.line_length_std.dropna()))*10**(-2)
length_pend_std=uncertainty_weighted(np.array(Data.lengh_pend_std.dropna()))*10**(-2)
line_length_std=uncertainty_weighted(np.array(Data.line_length_std.dropna()))*10**(-2)
tot_length=np.array([length_pend+line_length,length_pend_std+line_length_std])

In [21]:
L,g,T ,C= symbols("L, g, T,C")
dL,dg,dT= symbols("sigma_L, sigma_g, sigma_T")

In [22]:
g=L*(2*pi/T)**2
dg=sqrt((dL*g.diff(L))**2+(g.diff(T)*dT)**2)
lprint(latex(Eq(symbols('g'), g)))
lprint(latex(Eq(symbols('sigma_dg'), dg)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
# Turn expression into numerical functions 
fg = lambdify((L,T,DR,T1,T),g)
fdg = lambdify((a,DB,DR,T1,T2,da,dDB,dDr,dT),dg)
# Numerically evaluate expressions and print 
vg= fg(big_weighted_slope,big_ball,d_rail,angle*np.pi/180,(angle_180-angle)*np.pi/180)
vdg = fdg(big_weighted_slope,big_ball,d_rail,angle*np.pi/180,(angle_180-angle)*np.pi/180,big_uncertainty_slope,big_ball_std,d_rail_std,angle_std*np.pi/180)
lprint(fr'q_0 = ({vg:} \pm {vdg:})\,\mathrm{{m/s^2}}')
